## Read Smallcases

In [1]:
import numpy as np
import pandas as pd

In [36]:
id = "0025"
smallcase = pd.ExcelFile(f"data/smallcases/SCNM_{id}_Timeline.xlsx")

constituents = pd.read_excel(smallcase, sheet_name="Historical Constituents")
constituents.head()

,Date Range,Constituents,Weightage
0,2017-11-15 to 2017-11-30,JBM Auto Ltd,0.08
1,NaN,Igarashi Motors India Ltd,0.08
2,NaN,Mahindra and Mahindra Ltd,0.08
3,NaN,Tata Motors Ltd,0.08
4,NaN,Hero MotoCorp Ltd,0.08


In [37]:
indexes = pd.read_excel(smallcase, sheet_name="Historical Index Values")
indexes.head()

,Date,Electric Mobility,NIFTY Large & Midcap,Rebalance Occured
0,2017-11-15,100.00,100.00,NaN
1,2017-11-16,101.03,101.08,NaN
2,2017-11-17,101.62,101.91,NaN
3,2017-11-20,102.26,102.32,NaN
4,2017-11-21,102.52,102.55,NaN


### Map and POST Smallcase Name to Upstox Instrument

In [23]:
# Connect to db
import os
import sys

backend_path = os.path.abspath(os.path.join(os.getcwd(), '../backend'))
sys.path.append(backend_path)
from app.internal.firebase import db

In [42]:
postable_constituents = constituents["Constituents"].unique()
len(postable_constituents)

30

In [95]:
async def query_ticker_name(name: str):
    return await db.collection("tickers").where("name", ">=", name).where("name", "<=", name + '\uf8ff').get()

In [96]:
successful_constitents = []
failed_constitents = []


for c in postable_constituents:
    q = c.upper()
    docs = await query_ticker_name(q)
    if len(docs) == 0:
        failed_constitents.append(c)
    else:
        for e in docs:
            await db.collection("tickers").document(e.id).update({
                "smallcase_name": c
            })
            successful_constitents.append((c, e.id))

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:302: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
/var/folders/4x/3kzzlsjs1q36v35r81960pt40000gn/T/ipykernel_64588/2276505340.py:2: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return await db.collection("tickers").where("name", ">=", name).where("name", "<=", name + '\uf8ff').get()


In [97]:
len(successful_constitents)


10

10 successful, 20 failed, wrangle and try again

In [108]:
# All return upper

def replace_ltd(s):
    return s.upper().replace("LTD", "LIMITED")

def replace_technologies(s):
    return s.upper().replace("TECHNOLOGIES", "TECHNO.")

def replace_company(s):
    return s.upper().replace("COMPANY", "CO")

def replace_corporation(s):
    return s.upper().replace("CORPORATION", "CORP")

In [130]:
# Ltd -> Limited
for f in failed_constitents:
    q = replace_ltd(f)
    docs = await query_ticker_name(q)
    if len(docs) != 0:
        for e in docs:
            await db.collection("tickers").document(e.id).update({
                "smallcase_name": f
            })
            successful_constitents.append((f, e.id))
            failed_constitents.remove(f)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/google/cloud/firestore_v1/base_collection.py:302: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
/var/folders/4x/3kzzlsjs1q36v35r81960pt40000gn/T/ipykernel_64588/2276505340.py:2: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return await db.collection("tickers").where("name", ">=", name).where("name", "<=", name + '\uf8ff').get()


In [131]:
len(successful_constitents), len(failed_constitents)

(19, 11)

In [116]:
nse = pd.read_csv("data/upstox/NSE.csv")
equities = nse.query('instrument_type == "EQUITY" and exchange == "NSE_EQ"')
equities.head()

,instrument_key,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,option_type,exchange
77,NSE_EQ|DUMMYSAN005,14747.0,011NSETEST,011NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
78,NSE_EQ|DUMMYSAN006,14751.0,021NSETEST,021NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
79,NSE_EQ|DUMMYSAN007,14753.0,031NSETEST,031NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
80,NSE_EQ|DUMMYSAN008,14755.0,041NSETEST,041NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ
81,NSE_EQ|DUMMYSAN009,14758.0,051NSETEST,051NSETEST,NaN,NaN,NaN,0.05,1.0,EQUITY,NaN,NSE_EQ


In [134]:
import difflib
df = equities.filter(['name', "exchange_token"])
tickers = pd.DataFrame() 
r = {}
for f in failed_constitents:
    df['similarity'] = df["name"].apply(lambda x: difflib.SequenceMatcher(None, x.lower(), f.lower()).ratio())
        
    top_match = df.sort_values(by='similarity', ascending=False).head(1)
    top_match['smallcase_name'] = f
        
    tickers = pd.concat([tickers, top_match], ignore_index=True)

tickers

,name,exchange_token,similarity,smallcase_name
0,MAHINDRA & MAHINDRA LTD,2031.0,0.916667,Mahindra and Mahindra Ltd
1,MINDA CORPORATION LTD,25897.0,0.851064,Indian Oil Corporation Ltd
2,HIMADRI SPECIALITY CHEM L,14334.0,0.892857,Himadri Speciality Chemical Ltd
3,HINDALCO INDUSTRIES LTD,1363.0,0.958333,Hindalco Industries Ltd
4,AMARA RAJA ENERGY MOB LTD,100.0,0.877193,Amara Raja Energy & Mobility Ltd
5,POWER GRID CORP. LTD.,14977.0,0.678571,Power Grid Corporation of India Ltd
6,ENDURANCE TECHNO. LTD.,18822.0,0.833333,Endurance Technologies Ltd
7,BHARAT ELECTRONICS LTD,383.0,0.800000,Bharat Heavy Electricals Ltd
8,TATA POWER CO LTD,3426.0,0.871795,Tata Power Company Ltd
9,SAMVRDHNA MTHRSN INTL LTD,4204.0,0.781250,Samvardhana Motherson International Ltd


In [137]:
payload = tickers[['exchange_token', 'smallcase_name']].to_dict(orient='records')
for p in payload:
    id = str(int(p['exchange_token']))
    await db.document(f"tickers/{id}").update({"smallcase_name": p["smallcase_name"]})
    successful_constitents.append((p["smallcase_name"], id))

In [138]:
len(successful_constitents)

30